## Enriching LULC with data on protected areas

This block of preprocessing code is dedicated to refining initial land-use/land-cover (LULC) data with additional data on protected areas (PA) from the World Database on Protected Areas (WDPA): https://www.protectedplanet.net/en/thematic-areas/wdpa
As soon as protected areas may significantly reduce the reflectance of landscapes for species migration, landscapes intersected with PAs should be considered as different from those with no protected status. This workflow is describing the process of updating LULC data needed to compute functional landscape connectivity. Impedance and affinity values derived from LULC data and required by Miramon and Graphab are also recomputed.

##### 1. Extracting data through WDPA API

Spatial data on protected areas in GeoJSON and GeoPackage formats for Spain, France and Andorra are obtained through WDPA API using a personal access token and official docimentation: https://api.protectedplanet.net/documentation. Most meaningful attributes have been chosen (IDs, designation status, IUCN category, year of establishment etc.)

In [12]:
import requests
from shapely.geometry import shape
import json
import subprocess
import os
from datetime import datetime

# 1. BUILD API REQUEST
## define the API endpoint - include filter by country, avoid marine areas, maximum values of protected areas per page (50)
api_url = "https://api.protectedplanet.net/v3/protected_areas/search?token={token}&country={country}&marine=false&with_geometry=true&per_page=50"
## define token - replace by own
token = "968cef6f0c37b925225fb60ac8deaca6"
## define country codes
countries = ["ESP", "FRA", "AND"]
## TODO - country codes should derive from the extent of buffered LULC data

## directory to save GeoJSON files
response_dir = "response"
os.makedirs(response_dir, exist_ok=True)
## list to store the names of the GeoJSON files
geojson_files = []

# 2. LOOP OVER COUNTRIES NEEDED

## loop over each ISO code
for country in countries:
    ## make GET request to the WDPA API
    url = api_url.format(country=country, token=token)
    response = requests.get(url)
    
    '''
    # to check content
    # print(response.content)
    '''
    
    ## check if the request was successful
    if response.status_code == 200:
        ## extract protected areas if they exist in the response
        response_json = response.json()
        protected_areas = response_json.get('protected_areas', [])

        ## create GeoJSON feature collection
        feature_collection = {
            "type": "FeatureCollection",
            "features": []
        }

        ## loop over protected areas        
        for pa in protected_areas:

            ## convert date string to datetime object
            date_str = pa.get('legal_status_updated_at')

            ## filter out protected areas if no date of establishment year is recorded
            if date_str:
                try:
                    date_obj = datetime.strptime(date_str, "%d/%m/%Y")
                    formatted_date = date_obj.strftime("%Y-%m-%d")
                except ValueError:
                    formatted_date = None
            else:
                formatted_date = None

            ## skip features with no year
            if not formatted_date:
                continue

            ## extract geometry
            geometry = pa.get('geojson', {}).get('geometry')

            ## debugging, print the geometry data
            if geometry is None:
                print(f"Warning: No geometry found for protected area {pa.get('name')} with ID {pa.get('id')}")
            else:
                print(f"Geometry found for protected area {pa.get('name')} with ID {pa.get('id')}")           
            
            '''
            # TO RUN TRANSFORMATION OF DATE INTO YEAR ONLY
            if date_str is None:
                year = None
            else:
                date_obj = datetime.strptime(date_str, "%d/%m/%Y")
                # extract year from datetime object
                year = date_obj.year
            '''

            ## create feature with geometry and properties
            feature = {
                "type": "Feature",
                "geometry": pa.get('geojson', {}).get('geometry'),
                "properties": {
                    "id": pa.get('id'),
                    "name": pa.get('name'),
                    "original_name": pa.get('name'),
                    "wdpa_id": pa.get('id'),
                    "management_plan": pa.get('management_plan'),
                    "is_green_list": pa.get('is_green_list'),
                    "iucn_category": pa.get('iucn_category'),
                    "designation": pa.get('designation'),
                    "legal_status": pa.get('legal_status'),
                    "year": pa.get('legal_status_updated_at')
                }
            }

            '''
            # debugging for features
            print (feature)
            '''
            ## append the feature to the feature collection
            feature_collection["features"].append(feature)
            
            '''
            ## debugging statement on printing protected areas
            print(f"Selected attributes for protected area {pa.get('name')} extracted successfully.")
            '''

        ## define filename for GeoJSON file
        geojson_filename = os.path.join(response_dir, f"{country}_protected_areas.geojson")
        ## convert GeoJSON data to a string
        geojson_string = json.dumps(feature_collection, indent=4) 
        ## write GeoJSON string to a file
        with open(geojson_filename, 'w') as f:
            f.write(geojson_string)
        
        print(f"GeoJSON data for {country} saved to {geojson_filename}")
        
        ## add the GeoJSON filename to the list
        geojson_files.append(geojson_filename)
    else:
        print(f"Error fetching data for {country}")

## define function to ensure the 'year' is formatted correctly
def format_year_attribute(geojson_file):
    with open(geojson_file, 'r') as f:
        data = json.load(f)

    for feature in data['features']:
        year_str = feature['properties'].get('year', None)
        if year_str:
            try:
                date_obj = datetime.strptime(year_str, "%d/%m/%Y")
                formatted_date = date_obj.strftime("%Y-%m-%d")
                feature['properties']['year'] = formatted_date
            except ValueError:
                feature['properties']['year'] = None
        else:
            feature['properties']['year'] = None
    
    with open(geojson_file, 'w') as f:
        json.dump(data, f, indent=4)

# 3. EXPORT TO GEOPACKAGE

## define the filename for the GeoPackage
gpkg = os.path.join(response_dir, "merged_protected_areas.gpkg")
## remove GeoPackage if it already exists
if os.path.exists(gpkg):
    os.remove(gpkg)

## loop through the GeoJSON files and convert them to a geopackage
for geojson_file in geojson_files:
    ## ensure the 'year' attribute is correctly formatted
    format_year_attribute(geojson_file)

    ## writes layer name as the first name from geojson files
    layer_name = os.path.splitext(os.path.basename(geojson_file))[0]
    ## use ogr2ogr to convert GeoJSON to GeoPackage
    subprocess.run([
        "ogr2ogr", "-f", "GPKG", "-append", "-nln", layer_name, gpkg, geojson_file
    ])

print(f"All GeoJSON data merged and saved to {gpkg}")

Geometry found for protected area Marismas del Río Palmones with ID 890
Geometry found for protected area Ordesa y Monte Perdido with ID 893
Geometry found for protected area Timanfaya with ID 895
Geometry found for protected area Garajonay with ID 897
Geometry found for protected area Torcal de Antequera with ID 4740
Geometry found for protected area Sierra Espuña with ID 4745
Geometry found for protected area Tablas de Daimiel with ID 4750
Geometry found for protected area Monfragüe with ID 4820
Geometry found for protected area Montaña de Riaño y Mampodre with ID 4840
Geometry found for protected area El Teide with ID 10196
Geometry found for protected area Garajonay National Park with ID 12206
Geometry found for protected area Itxusi with ID 13350
Geometry found for protected area Castala with ID 13364
Geometry found for protected area Isla de Enmedio with ID 15376
Geometry found for protected area Complejo Endorreico de Chiclana with ID 15378
Geometry found for protected area Mari

##### 2. Processing of protected areas

Data downloaded from WDPA as geopackage are [processed](./pas_timeseries.py) in 4 steps:
1. Extract extent and spatial resolution of LULC data.
Redefine no data values as 0 for input LULC data.
2. Extract protected areas filtered by LULC timestamp and year of PAs establishment. As WDPA data fetching is limited by 50 features per response page, this part of code uses data downloaded not through WDPA API but through unauthorised access from WDPA website (CSV transformed into GeoPackage).
3. Rasterize protected areas (there is no way to read geodataframes by gdal_rasterize except from writing files on the disc) based on step 1.
4. Compress protected areas.

In [13]:
import geopandas as gpd
import rasterio
import os
import subprocess

# load geopackage with protected areas
gdf = gpd.read_file(r"response/pas_upd.gpkg")
# to check column names use:
# print(gdf.columns)

# define input folder
input_folder = r'lulc'
# assign output folder
output_dir = ('pas_timeseries')
# create output folder if it doesn't exist - only needed for exporting as gpkgs
os.makedirs(output_dir, exist_ok=True)

It is important to extract year stamps.

In [14]:
## list all TIFF files in input folder
tiff_files = [f for f in os.listdir(input_folder) if f.endswith('.tif')]
## extract year stamps from filenames (removes the first part before _ and the part after .)
year_stamps = [int(f.split('_')[1].split('.')[0]) for f in tiff_files]
print("Considered timestamps of LULC data are:",year_stamps)

Considered timestamps of LULC data are: [1987, 1992, 1997, 2002, 2007, 2012, 2017, 2022]


Then, extent of LULC files (minimum and maximum coordinates) is extracted.

In [15]:
## define function
def extract_ext_res(file_path):
    with rasterio.open(file_path) as src:
        extent = src.bounds
        res = src.transform[0]  # assuming the res is the same for longitude and latitude
    return extent, res

## execute function
if tiff_files:
    file_path = os.path.join(input_folder, tiff_files[0])  # choose the first TIFF file (it shouldn't matter which LULC file to extract extent because they must have the same extent)
    extent, res = extract_ext_res(file_path)
    min_x = extent.left
    max_x = extent.right
    min_y = extent.bottom
    max_y = extent.top
    
    print("Extent of LULC files")
    print("Minimum X Coordinate:", min_x)
    print("Maximum X Coordinate:", max_x)
    print("Minimum Y Coordinate:", min_y)
    print("Maximum Y Coordinate:", max_y)
    print("Spatial resolution (pixel size):", res)
else:
    print("No LULC files found in the input folder.")

# TODO - redefine null values from LULC data as 0 or something else?

Extent of LULC files
Minimum X Coordinate: 230205.0
Maximum X Coordinate: 556485.0
Minimum Y Coordinate: 4459725.0
Maximum Y Coordinate: 4777335.0
Spatial resolution (pixel size): 30.0


Protected areas should be filtered by year stamp according to the PA's establishment year.

In [16]:
# create an empty dictionary to store subsets
subsets_dict = {}
# loop through each year_stamp and create subsets
for year_stamp in year_stamps:
    # filter Geodataframe based on the year_stamp
    subset = gdf[gdf['STATUS_YR'] <= year_stamp]
    
    # store subset in the dictionary with year_stamp as key
    subsets_dict[year_stamp] = subset

    # print key-value pairs of subsets 
    print(f"Protected areas are filtered according to year stamps of LULC and PAs' establishment year: {year_stamp}")

    # ADDITIONAL BLOCK IF EXPORT TO GEOPACKAGE IS NEEDED (currently needed as rasterizing vector data is not possible with geodataframes)
    ## save filtered subset to a new GeoPackage
    subset.to_file(os.path.join(output_dir,f"pas_{year_stamp}.gpkg"), driver='GPKG')
    print(f"Filtered protected areas are written to:",os.path.join(output_dir,f"pas_{year_stamp}.gpkg"))

print ("---------------------------")

Protected areas are filtered according to year stamps of LULC and PAs' establishment year: 1987
Filtered protected areas are written to: pas_timeseries\pas_1987.gpkg
Protected areas are filtered according to year stamps of LULC and PAs' establishment year: 1992
Filtered protected areas are written to: pas_timeseries\pas_1992.gpkg
Protected areas are filtered according to year stamps of LULC and PAs' establishment year: 1997
Filtered protected areas are written to: pas_timeseries\pas_1997.gpkg
Protected areas are filtered according to year stamps of LULC and PAs' establishment year: 2002
Filtered protected areas are written to: pas_timeseries\pas_2002.gpkg
Protected areas are filtered according to year stamps of LULC and PAs' establishment year: 2007
Filtered protected areas are written to: pas_timeseries\pas_2007.gpkg
Protected areas are filtered according to year stamps of LULC and PAs' establishment year: 2012
Filtered protected areas are written to: pas_timeseries\pas_2012.gpkg
Prot

Rasterization function based on yearstamps of protected areas is launched.

In [17]:
## list all subsets of protected areas by the year of establishment
pas_yearstamps = [f for f in os.listdir(output_dir) if f.endswith('.gpkg')]
pas_yearstamp_rasters = [f.replace('.gpkg', '.tif') for f in pas_yearstamps]

# loop through each input file
for pas_yearstamp, pas_yearstamp_raster in zip(pas_yearstamps, pas_yearstamp_rasters):
    pas_yearstamp_path = os.path.join(output_dir, pas_yearstamp)
    pas_yearstamp_raster_path = os.path.join(output_dir, pas_yearstamp_raster)
    # TODO - to make paths more clear and straightforward

    # rasterize
    pas_rasterize = [
        "gdal_rasterize",
        ##"-l", "pas__merged", if you need to specify the layer
        "-burn", "100", ## assign code starting from "100" to all LULC types
        "-init", "0",
        "-tr", str(res), str(res), #spatial res from LULC data
        "-a_nodata", "-2147483647", # !DO NOT ASSIGN 0 values with non-data values as it will mask them out in raster calculator
        "-te", str(min_x), str(min_y), str(max_x), str(max_y), # minimum x, minimum y, maximum x, maximum y coordinates of LULC raster
        "-ot", "Int32",
        "-of", "GTiff",
        "-co", "COMPRESS=LZW",
        pas_yearstamp_path,
        pas_yearstamp_raster_path
        ]

    # execute rasterize command
    try:
        subprocess.run(pas_rasterize, check=True)
        print("Rasterizing of protected areas has been successfully completed for", pas_yearstamp)
    except subprocess.CalledProcessError as e:
        print(f"Error rasterizing protected areas: {e}")

Rasterizing of protected areas has been successfully completed for pas_1987.gpkg
Rasterizing of protected areas has been successfully completed for pas_1992.gpkg
Rasterizing of protected areas has been successfully completed for pas_1997.gpkg
Rasterizing of protected areas has been successfully completed for pas_2002.gpkg
Rasterizing of protected areas has been successfully completed for pas_2007.gpkg
Rasterizing of protected areas has been successfully completed for pas_2012.gpkg
Rasterizing of protected areas has been successfully completed for pas_2017.gpkg
Rasterizing of protected areas has been successfully completed for pas_2022.gpkg


##### 3. Raster algebra

LULC [enriched](/raster_loop.sh) through the raster calculator (currently, external shell script):
1. Rearranging no data values as they must be considered as 0 to run raster calcualtions.
2. To sum initial LULC raster and protected areas (according to the timestamp).
3. Writing the new updated LULC map with the doubled amount of LULC codes for each timestamp (loop based on year matching in filenames).
4. Compression and assignment of null values.

##### 4. Updating landscape impedance
Impedance is reclassified by [CSV table](/reclassification.csv) and compressed (through LZW compression, not Cloud Optimised Geotiff format to avoid any further issues in Graphab). Landscape impedance is required by Miramon ICT and Graphab tools both.

In [7]:
from osgeo import gdal
import numpy as np
import csv
import os
import subprocess
gdal.UseExceptions()

# specify function to reclassify LULC by mapping dictionary and obtaining impedance raster data
def reclassify_raster(input_raster, output_raster, reclass_table):
    # read reclassification table
    reclass_dict = {}
    with open(reclass_table, 'r', encoding='utf-8-sig') as csvfile:  # handle UTF-8 with BOM
        reader = csv.DictReader(csvfile)
        # initialize a flag to indicate if any row contains decimal values
        has_decimal_values = False
        
        next(reader, None) # skip headers for looping
        for row in reader:
            try:
                impedance_rounded_str = row['impedance']
                if '.' in impedance_rounded_str:  # check if impedance contains decimal values
                    has_decimal_values = True
                    break  # exit the loop if any row contains decimal values
            except ValueError:
                print("Invalid data format in reclassification table.")
            continue

        # reset file pointer to read from the beginning
        csvfile.seek(0)

        # read classification table again and define mapping for decimal and integer values
        next(reader, None) # skip headers for looping
        if has_decimal_values:
            data_type = 'Float32'
            for row in reader:
                try:
                    lulc = int(row['lulc'])
                    impedance = float(row['impedance'])
                    reclass_dict[lulc] = impedance
                except ValueError:
                    print("Invalid data format in reclassification table_2. Problematic row:", row)
                    continue
        else:
            data_type = 'Int32'
            for row in reader:
                try:
                    lulc = int(row['lulc'])
                    impedance = int(row['impedance'])
                    reclass_dict[lulc] = impedance
                except ValueError:
                    print("Invalid data format in reclassification table_3.")
                    continue
  
        if has_decimal_values:
            print("LULC impedance is characterized by decimal values.")
            # update reclassification dictionary to align nodata values with one positive value (Graphab requires positive value as no_data value)
            # assuming nodata value is 9999 (or 9999.00 if estimating decimal values)
            reclass_dict.update({-2147483647: 9999.00, -32768: 9999.00, 0: 9999.00}) # minimum value for int16, int32 and 0 are assigned with 9999.00 (nodata)
        else:
            print("LULC impedance is characterized by integer values only.")
            # update dictionary again
            reclass_dict.update({-2147483647: 9999, -32768: 9999, 0: 9999}) # minimum value for int16, int32 and 0 are assigned with 9999.00 (nodata)
    
    print ("Mapping dictionary used to classify impedance is:", reclass_dict)

    # open input raster
    dataset = gdal.Open(input_raster)
    if dataset is None:
        print("Could not open input raster.")
        return

    # get raster info
    cols = dataset.RasterXSize
    rows = dataset.RasterYSize

    # initialize output raster
    driver = gdal.GetDriverByName("GTiff")
    if has_decimal_values:
        output_dataset = driver.Create(output_raster, cols, rows, 1, gdal.GDT_Float32)
    else:
        output_dataset = driver.Create(output_raster, cols, rows, 1, gdal.GDT_Int32)
    #TODO - to add condition on Int32 if integer values are revealed
    output_dataset.SetProjection(dataset.GetProjection())
    output_dataset.SetGeoTransform(dataset.GetGeoTransform())

    # reclassify each pixel value
    input_band = dataset.GetRasterBand(1)
    output_band = output_dataset.GetRasterBand(1)
    # read the entire raster as a NumPy array
    input_data = input_band.ReadAsArray()

    # apply reclassification using dictionary mapping
    output_data = np.vectorize(reclass_dict.get)(input_data)
    output_band.WriteArray(output_data)

    '''FOR CHECKS
    print (f"input_data_shape is': {input_data.shape}")
    print (f"output_data_shape is': {output_data.shape}")
    '''

    # close datasets
    dataset = None
    output_dataset = None

    return (data_type)

if __name__ == "__main__":
    input_folder = r'lulc_pa'
    output_folder = r'impedance_pa'
    reclass_table = "reclassification.csv"
    
    # list all TIFF files in input folder
    tiff_files = [f for f in os.listdir(input_folder) if f.endswith('.tif')]
    # create output folder if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)
    # loop through each input file
    for tiff_file in tiff_files:
        input_raster_path = os.path.join(input_folder, tiff_file)
        print (tiff_file)
        # modify the output raster filename to ensure it's different from the input raster filename
        output_filename = "impedance_" + tiff_file
        output_raster_path = os.path.join(output_folder, output_filename)

        # call function and capture data_type for compression - Float32 or Int32
        data_type = reclassify_raster(input_raster_path, output_raster_path, reclass_table)    
        print ("Data type used to reclassify LULC as impedance is",data_type) 
        
        # compression using 9999 as nodata
        compressed_raster_path = os.path.splitext(output_raster_path)[0] + '_compr.tif'
        subprocess.run(['gdal_translate', output_raster_path, compressed_raster_path,'-a_nodata', '9999', '-ot', data_type, '-co', 'COMPRESS=LZW'])

        # as soon as gdal_translate doesn't support rewriting, we should delete non-compressed GeoTIFFs...
        os.remove(output_raster_path)
        # ...and rename compressed file in the same way as the original GeoTIFF
        os.rename(compressed_raster_path, output_raster_path)

        print("Reclassification complete for:", input_raster_path + "\n------------------------------------")


lulc_1987_pa.tif
LULC impedance is characterized by decimal values.
Mapping dictionary used to classify impedance is: {1: 4.0, 2: 1000.0, 3: 5.7, 4: 3.4, 5: 2.7, 6: 1.0, 7: 2.7, 101: 2.0, 102: 500.0, 103: 2.85, 104: 1.7, 105: 1.35, 106: 0.5, 107: 1.35, -2147483647: 9999.0, -32768: 9999.0, 0: 9999.0}
Data type used to reclassify LULC as impedance is Float32
Reclassification complete for: lulc_pa\lulc_1987_pa.tif
------------------------------------
lulc_1992_pa.tif
LULC impedance is characterized by decimal values.
Mapping dictionary used to classify impedance is: {1: 4.0, 2: 1000.0, 3: 5.7, 4: 3.4, 5: 2.7, 6: 1.0, 7: 2.7, 101: 2.0, 102: 500.0, 103: 2.85, 104: 1.7, 105: 1.35, 106: 0.5, 107: 1.35, -2147483647: 9999.0, -32768: 9999.0, 0: 9999.0}
Data type used to reclassify LULC as impedance is Float32
Reclassification complete for: lulc_pa\lulc_1992_pa.tif
------------------------------------
lulc_1997_pa.tif
LULC impedance is characterized by decimal values.
Mapping dictionary used to c

##### 5. Updating landscape affinity 
Landscape affinity is computed and compressed based on the math expression processing landscape impedance. By now (04/06/2024), landscape affinity is computed as a reversed value of landscape impedance but it is planned to develop it as a more flexible input to compute connectivity further. This output is required by Miramon ICT software, not Graphab.

In [8]:
impedance_dir = 'impedance_pa'
affinity_dir = 'affinity'
# create the affinity directory if it doesn't exist
if not os.path.exists(affinity_dir):
    os.makedirs(affinity_dir)

impedance_files = os.listdir(impedance_dir)
print (impedance_files)

# loop through each TIFF file in impedance_dir
for impedance_file in impedance_files:
    if impedance_file.endswith('.tif'):
        # construct full paths for impedance and affinity files
        impedance_path = os.path.join(impedance_dir, impedance_file)
        affinity_path = os.path.join(affinity_dir, impedance_file.replace('impedance', 'affinity'))

        # open impedance file
        ds = gdal.Open(impedance_path)

        if ds is None:
            print(f"Failed to open impedance file: {impedance_file}")
            continue

        # get raster band
        band = ds.GetRasterBand(1)
        # read raster band as a NumPy array
        data = band.ReadAsArray()
        # reverse values with condition (if it is 9999 leave it, otherwise make it reversed)
        reversed_data = np.where(data == 9999, data, 1 / data)

        # write reversed data to affinity file
        driver = gdal.GetDriverByName("GTiff")
        out_ds = driver.Create(affinity_path, ds.RasterXSize, ds.RasterYSize, 1, gdal.GDT_Float32)
        out_ds.GetRasterBand(1).WriteArray(reversed_data)

        # copy georeferencing info
        out_ds.SetGeoTransform(ds.GetGeoTransform())
        out_ds.SetProjection(ds.GetProjection())

        # close files
        ds = None
        out_ds = None

        print(f"Affinity computed for: {impedance_file}")

        # compression
        compressed_raster_path = os.path.splitext(affinity_path)[0] + '_compr.tif'
        subprocess.run(['gdal_translate', affinity_path, compressed_raster_path,'-a_nodata', '9999', '-ot', 'Float32', '-co', 'COMPRESS=LZW'])
    
        # as soon as gdal_translate doesn't support rewriting, we should delete non-compressed GeoTIFFs...
        os.remove(affinity_path)
        # ...and rename COG in the same way as the original GeoTIFF
        os.rename(compressed_raster_path, affinity_path)
        print(f"Affinity file is successfully compressed.", end="\n------------------------------------------\n")

print("All LULC affinities have been successfully computed.")

['impedance_lulc_1987_pa.tif', 'impedance_lulc_1992_pa.tif', 'impedance_lulc_1997_pa.tif', 'impedance_lulc_2002_pa.tif', 'impedance_lulc_2007_pa.tif', 'impedance_lulc_2012_pa.tif', 'impedance_lulc_2017_pa.tif', 'impedance_lulc_2022_pa.tif']
Affinity computed for: impedance_lulc_1987_pa.tif
Affinity file is successfully compressed.
------------------------------------------
Affinity computed for: impedance_lulc_1992_pa.tif
Affinity file is successfully compressed.
------------------------------------------
Affinity computed for: impedance_lulc_1997_pa.tif
Affinity file is successfully compressed.
------------------------------------------
Affinity computed for: impedance_lulc_2002_pa.tif
Affinity file is successfully compressed.
------------------------------------------
Affinity computed for: impedance_lulc_2007_pa.tif
Affinity file is successfully compressed.
------------------------------------------
Affinity computed for: impedance_lulc_2012_pa.tif
Affinity file is successfully comp